# LangChain Chat Part 2 - Conversation

From DeepLearning.ai's `LangChain Chat with Your Data` course.

Tutorial: https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/7/chat

Be sure to persist vector store to disk by running langchain_chat_1_persist_vectore_store notebook before running this one.

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key  = os.environ['OPENAI_API_KEY']

## Load Vectore Store from Filesystem

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

## Build Memory

In [3]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever=vectordb.as_retriever()

# ConversationRetrievalChain adds a new step on top of the RetrievalQA chain.
# It takes the history and the new question, and condenses it into a standalone question to pass to the vector store to look up relevant documents.
# It also uses this condensed standalone question to pass to the language model to get an answer along with the relevant chunks from the vector store.
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)

In [5]:
question = "Is probability a class topic?"
result = qa({"question": question})
print(result["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
of this class will not be very program ming intensive, although we will do some 
programming, mostly in either MATLAB or Octa ve. I'll say a bit more about that later.  
I also assume familiarity with basic proba bility and statistics. So most undergraduate 
statistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna 
assume all of you know what ra ndom variables are, that all of you know what expectation 
is, what a variance or a random variable is. And in case of some of you, it's been a while 
since you've seen some of this material. At some of the discussion sections, we'll actually 
go over some of the prerequisites, sort of as  a refresher course under prerequisi

In [6]:
question = "why are those prerequesites needed?"
result = qa({"question": question})
print(result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is probability a class topic?
Assistant: Yes, probability is a topic that will be covered in the class. The instructor mentions assuming familiarity with basic probability and statistics, and also mentions using a probabilistic interpretation to derive a learning algorithm.
Follow Up Input: why are those prerequesites needed?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
So, see, most of us use learning algorithms half a dozen, a dozen, maybe dozens of times 
without even knowing it